In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [ ]:
# RuntimeError: stack expects each tensor to be equal size, but got [43] at entry 0 and [30] at entry 1
# 이 에러는 DataLoader가 배치를 만들 때 각 샘플의 길이가 달라서 발생했다.
# LSTM 모델에 입력할 데이터는 일정한 길이의 시퀀스로 맞춰줘야 한다. 따라서 리뷰 텍스트의 길이를 맞추기 위해 패딩(padding)을 적용하는 방법이 필요하다.

# 즉, 길이가 서로 다른 리뷰를 동일한 길이로 맞춰주는 함수(=> collate_fn)를 정의하여 DataLoader에 전달해야 한다.

from torch.nn.utils.rnn import pad_sequence

# 패딩 적용 함수
def collate_fn(batch):
    reviews, ratings = zip(*batch)
    reviews_padded = pad_sequence(reviews, batch_first=True, padding_value=0)  # 0으로 패딩
    ratings = torch.tensor(ratings)  # 텐서로 변환
    return reviews_padded, ratings

# 데이터 로더 정의 시 collate_fn 전달
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

In [ ]:
# LSTM 모델 정의
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

# ValueError: Expected input batch_size (1) to match target batch_size (64).
# 이 오류는 outputs의 배치 크기와 y_batch의 배치 크기가 일치하지 않아서 발생한다. outputs와 y_batch 모두 64개의 배치 크기를 가져야 한다.
# 지금 모델의 forward 메서드에서 embedded.unsqueeze(0)을 사용해 배치 크기를 1로 만들고 있다.
# 이를 해결하려면 forward 메서드에서 배치 크기 전체를 처리할 수 있도록 unsqueeze(0)을 제거야 한다.
# 또는, nn.EmbeddingBag 대신 nn.Embedding을 사용할 수도 있다.

    def forward(self, text):
        embedded = self.embedding(text)    # (batch_size, seq_length, embed_dim)
        output, (hidden, cell) = self.lstm(embedded)   # 원래 self.lstm(embedded.unsqueeze(0))였는데, unsqueeze(0)을 제거했다.   # output: (batch_size, seq_length, hidden_dim)
        # hidden의 마지막 상태를 사용하여 출력을 만든다.
        return self.fc(hidden[-1])# hidden[-1]은 (1, batch_size, hidden_dim) 형태이므로, squeeze 필요

# RuntimeError: size mismatch (got input: [5], target: [64])
# 이번 오류는 모델의 출력 차원과 타겟(y_batch)의 차원이 맞지 않아서 발생했다.
# 주의해야 할 점은 outputs의 형상이 (batch_size, output_dim)이어야 하며, y_batch는 크기가 (batch_size,)인 텐서여야 한다는 것이다.
# 현재 오류 메시지에서 outputs의 크기가 [5]이고 y_batch의 크기가 [64]라는 것은, 모델이 배치의 모든 데이터에 대해 올바른 출력을 내지 못하고 있다는 것을 의미한다.
# 여기서 몇 가지 사항을 확인하고 수정해야 한다.
# 모델 출력 차원: output_dim이 올바르게 설정되었는지 확인하세요. 보통 감정 분류 문제에서는 클래스의 수와 같아야 합니다.
# 데이터 준비: 데이터 로더에서 배치 크기를 확인하고, y_batch가 올바른 형태로 준비되었는지 점검하세요.

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
# 학습 루프에서 y 대신 y_batch 사용
# y 사용 시 에러 발생 : TypeError: cross_entropy_loss(): argument 'target' (position 2) must be Tensor, not list

num_epochs = 100
for epoch in range(num_epochs):
    for X_batch, y_batch in train_dataloader:
        outputs = model(X_batch)
        optimizer.zero_grad()
        loss = criterion(outputs, y_batch)   # y_batch를 사용하여 텐서 형태 보장. 이렇게 하면 CrossEntropyLoss에 전달되는 target이 올바르게 텐서로 인식되어 에러가 해결된다.
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

print('Finished Training')